<a href="https://colab.research.google.com/github/six-ten/image-classification-notebooks/blob/master/GalasAutoTagChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# dataset is expected to be present in a folder called Datasets as dataset.zip in gdrive

In [0]:
from google.colab import drive
from google.colab import files
drive.mount('/content/gdrive')
dataset_path = '/content/gdrive/My Drive/Datasets/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np
import cv2
import pandas as pd
import os 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras import models,layers,applications, optimizers

 # creating a local copy of dataset for faster IO , loading 1000s of images over network directly from gdrive can be a bottleneck


In [0]:
os.chdir('/')

In [0]:
!mkdir LocalStorage

mkdir: cannot create directory ‘LocalStorage’: File exists


In [0]:
!cp -r 'content/gdrive/My Drive/Datasets/dataset.zip' '/LocalStorage/'

In [0]:
os.listdir('./LocalStorage/')


['dataset', 'dataset.zip']

In [0]:
!unzip -q "/LocalStorage/dataset.zip" -d "/LocalStorage/"

replace /LocalStorage/dataset/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
os.chdir("/LocalStorage/dataset/")

In [0]:
train_df = pd.read_csv('./train.csv')

In [0]:
test_df = pd.read_csv('./test.csv')

In [0]:
train_df.head()

In [0]:
test_df.head()

In [0]:
class_counts = dict(train_df['Class'].value_counts())
print(class_counts)


In [0]:
class_labels = {'Food': 0, 'Attire': 1, 'misc': 2, 'Decorationandsignage': 3}

# it can be seen that the dataset has unequal number of samples from each class, so we calculate weight factors to calculate weighted loss to compensate for imbalance in distribution

In [0]:
class_weights = {class_labels[key]: sum(class_counts.values())/class_counts[key] for key in class_counts.keys()}
print(class_weights)

In [0]:
# this dictionary will be used to map predictions with labels
predict_dict  = {value:key for key, value in class_labels.items()}
print(predict_dict)

# an unused function that is supposed to randomly sample and upsample dataset to account for imbalance in distribution, returns a dataframe with duplicated images, marked True for augment flag, images with that flag are duplicates and should be modified in some way (augmentation) if this method is used

In [0]:

def upsample_before_train(df):
  class_counts = dict(train_labels['Class'].value_counts())
  df_dict = filter_DataFrame(df)
  ref_count = max(list(class_counts.values()))
  for cls, df in df_dict.items():
        class_df = df_dict[cls]
        instance_count = class_df.shape[0]
        scale_factor = ref_count/instance_count
        for _ in range(int(scale_factor)-1):
          df_dict[cls] = df_dict[cls].append(class_df.sample(class_counts[cls]))
        df_dict[cls] = df_dict[cls].append(class_df.sample(int(class_counts[cls]*(scale_factor-int(scale_factor)))))
        df_dict[cls]['augment'] = df_dict[cls].duplicated()
        df_dict[cls] = shuffle(df_dict[cls])
  return df_dict

In [0]:
def smart_crop(img,thresh=0.8,mode='split'):
  
  h,w = img.shape[:-1]

  if min([w,h])/max(w,h) > thresh :
    return [img]
  else :
    if h>w :
      if mode == 'center':
        h2 = h//2
        w2 = w//2
        return [img[h2-w2:h2+w2,:]]
      return [img[:w,:],img[-w:]]
    else :
      if mode == 'center':
        w2 = w//2
        h2 = h//2
        return [img[:,w2-h2:w2+h2]]
      return [img[:,:h],img[:,-h:]]

In [0]:
def get_datav1(dfs,image_dir_names,output_shape=(224,224),images_only=False):
  '''functions specific to this example
  df : dataframe containing input data
  image_dir_name : name of folder containing images
  output_shape : output size of images after preprocessing
  images_only : flag to determine the output true -> returns images only -> for running predictions
                false -> return labels and images -> for training
  
   '''
  imgs = []
  lbls = []
  f = 2 ;
  if images_only :
    f = 1 
  for df,image_dir_name in zip(dfs,image_dir_names):
    
    df_dict = df.to_dict()
    images = df_dict['Image']
    if not images_only:
      labels = df_dict['Class']
    for key,img in images.items():
      
      Im = cv2.imread('./'+image_dir_name+'/'+img)
      Image_heights.append(Im.shape[0])
      Image_widths.append(Im.shape[1])
      Ims = smart_crop(Im)
      #Ims = [Im]
      Ims = [cv2.resize(x,output_shape) for x in Ims]
      for Im in Ims:
        imgs.append(Im)
        if not images_only:
          imgs.append(cv2.flip(Im,1))
      if not images_only :
        for _ in range(f*len(Ims)):
          lbls.append(class_labels[labels[key]])
  imgs = applications.vgg16.preprocess_input(np.array(imgs))
  if images_only :
    return imgs
  else :
    return imgs,np.array(lbls)

In [0]:
Image_heights = []
Image_widths = []
def get_data(dfs,image_dir_names,output_shape=(224,224),images_only=False):
  global Image_heights, Image_widths
  '''functions specific to this example
  df : dataframe containing input data
  image_dir_name : name of folder containing images
  output_shape : output size of images after preprocessing
  images_only : flag to determine the output true -> returns images only -> for running predictions
                false -> return labels and images -> for training
  
   '''
  imgs = []
  lbls = []
  f = 2 ;
  if images_only :
    f = 1 
  for df,image_dir_name in zip(dfs,image_dir_names):
    
    df_dict = df.to_dict()
    images = df_dict['Image']
    if not images_only:
      labels = df_dict['Class']
    for key,img in images.items():
      
      Im = cv2.imread('./'+image_dir_name+'/'+img)
      Image_heights.append(Im.shape[0])
      Image_widths.append(Im.shape[1])
      Ims = smart_crop(Im,mode='center')
      #Ims = [Im]
      Im = cv2.resize(Im,output_shape)
      imgs.append(Im)
      if not images_only :
        lbls.append(class_labels[labels[key]])
  imgs = applications.vgg16.preprocess_input(np.array(imgs))
  if images_only :
    return imgs
  else :
    return imgs,np.array(lbls)

In [0]:
#X_train, Y_train = get_data([train_df,predictions],['Train Images','Test Images'])
X_train, Y_train = get_datav1([train_df.sample(2000)],['Train Images'])

In [0]:
X_train.shape

In [0]:
Y_t_df = pd.DataFrame(Y_train)

In [0]:
Y_train_counts = Y_t_df[0].value_counts().to_dict()

In [0]:
Y_train_counts

In [0]:
aug_class_weights = {key: sum(Y_train_counts.values())/Y_train_counts[key] for key in Y_train_counts.keys()}
print(aug_class_weights)

In [0]:
X_predict = get_data([test_df],['Test Images'],images_only=True)
print(X_predict.shape)

In [0]:
#Y_train_ohv = tf.keras.utils.to_categorical(Y_train,len(class_labels))

In [0]:
def create_custom_model(input_shape=(80,80,3),n_classes=4):
  new_model = models.Sequential()
  new_model.add(layers.Input(shape=input_shape))
  new_model.add(layers.Conv2D(64,5))
  new_model.add(layers.Conv2D(128,5))
  new_model.add(layers.Conv2D(128,5))
  new_model.add(layers.MaxPool2D((2,2)))
  new_model.add(layers.BatchNormalization())
  new_model.add(layers.Conv2D(128,3))
  new_model.add(layers.Conv2D(128,3))
  new_model.add(layers.Flatten())
  new_model.add(layers.Dense(1024,activation='relu'))
  new_model.add(layers.Dense(n_classes,activation='softmax'))
  return new_model


In [0]:
def createTLmodel(x_train,y_train,n_classes = 4,fraction=0.9,params):
  
  net = applications.VGG16()
  new_model = models.Sequential()
  split_layer_index = int(fraction*len(net.layers))
  for i,layer in enumerate(net.layers[:-1]) :
    new_model.add(layer)
    if (i<split_layer_index):
      layer.trainable = False
      #pass
  '''new_model.add(layers.Dense(512,activation='relu'))
  new_model.add(layers.Dropout(0.5))
  new_model.add(layers.Dense(512,activation='relu'))
  new_model.add(layers.Dropout(0.5))'''
  new_model.add(layers.Dense(n_classes,params['activation']))
  new_model.compile(loss='categorical_crossentropy',optimizer=params['optimizer'])
  out = new_model.fit(x_train,y_train,batch_size=params['batch_size'],epochs=params['epochs']
                      validation_split = params['validation_split']
                      )
  return out,new_model
  


In [0]:
def unfreeze_from_last(model):
  for layer in model.layers[::-1]:
    if not layer.trainable :
      layer.trainable = True
      break

In [0]:
def freeze_from_last(model):
  for layer in model.layers:
    if layer.trainable == True :
      layer.trainable = False
      break


In [0]:
def create_model(act,opt,lr,n_classes=4):
  opts = {
      'adam':optimizers.Adam(lr),
      'sgd':optimizers.SGD(lr)
  }
  opt = opts[opt]
  net = applications.VGG16()
  new_net = models.Sequential()
  for layer in net.layers[:-1]:
    layer.trainable = False
    new_net.add(layer)
    
  new_net.add(layers.Dense(4,activation=act))
  unfreeze_from_last(new_net)
  unfreeze_from_last(new_net)
  new_net.compile(loss='sparse_categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
  return new_net
  

In [0]:
evaluations=[]
done = set()
def Scan(data_x,data_y,model_function,params,validation_split=0.2):
  global evaluations,done
  
  evaluations = []
  l = int(validation_split*len(data_x))
  train_x = data_x[:l]
  train_y = data_x[:l]
  val_x = data_x[l:]
  val_y = data_y[l:]
  activations  =params['activation']
  optimizers = params['optimizers']
  learning_rates = params['learning_rates']
  for act in activations :
    for opt in optimizers :
      for lr in learning_rates :
        
        eval_key = opt+act+str(lr)
        if eval_key in done :
          continue
        print('training config : ',opt,' ',act,' ',lr)
        model = model_function(act,opt,lr)
        result = model.fit(data_x,data_y,batch_size=16,epochs=3,class_weight=aug_class_weights,validation_split=validation_split)
        tf.reset_default_graph()
        tf.keras.backend.clear_session()
        evaluations.append([act,opt,lr,result])
        done.add(eval_key)
        print(result)
  return evaluations

In [0]:
parameters = {
    'optimizers':['adam','sgd'],
    'learning_rates':[0.001,0.0001,0.01,0.1],
    'activation':['softmax']
}

In [0]:
evals = Scan(X_train[:2000],Y_train[:2000],create_model,parameters)

In [0]:
tf.reset_default_graph()

In [0]:
def generate_prediction_csv(my_model=my_model):
  res = my_model.predict(X_predict)
  res = np.argmax(res,axis=1)
  new_df = test_df.copy(deep=True)
  new_df['Class'] = [predict_dict[x] for x in res]
  new_df.to_csv('/content/gdrive/My Drive/Datasets/prediction3.csv',index=False)
  return new_df


In [0]:
tf.keras.backend.clear_session()

In [0]:
tf.get_default_graph()

In [0]:
e1 = evaluations[0][-1]

In [0]:
e1

In [0]:
!nvidia-smi

In [0]:
!ps -aux|grep python

In [0]:
!kill -9 24 772 1267 1269

In [0]:
e1.history